In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

In [3]:
def computeInterCluster(R):
    iloczynySkalarne = -2*np.dot(R.T, R)
    kwadratyDlugosci = np.sum(R**2, axis=0, keepdims=True)
    iloczynySkalarne += kwadratyDlugosci
    iloczynySkalarne += kwadratyDlugosci.T

    return iloczynySkalarne

def computeIntraClusterDistances(data, C, R, I, K, N):
    swojeCentra = np.take(R, C, axis=1)
    swojeCentra -= data
    swojeCentra **= 2

    intraClusterDistances = np.sqrt(np.sum(swojeCentra, axis=0, keepdims=True))

    liczebnosc = np.zeros(K)
    intraAvg = np.zeros(K)
    intraMax = np.zeros(K)
    
    for i in range(N):
        intraAvg[C[i]] += intraClusterDistances[0][i]
        intraMax[C[i]] = max(intraClusterDistances[0][i], intraMax[C[i]])
        liczebnosc[C[i]] += 1
    
    intraAvg /= np.maximum(liczebnosc, 1)
    
    return (intraAvg, intraMax)
        
    

def KMeans(K, data, printInfo=False):
    d = np.size(data, axis=0)
    N = np.size(data, axis=1)
    I = np.eye(K)
    
    #Centra grup (losowe wektory z danych)
    #R = np.take(data, np.random.choice(np.arange(0,N), K, replace=False), axis=1)
    R = np.empty([d,K])
    choices = np.random.choice(np.arange(0,N), K, replace=False)
    for i in range(K):
        R[:,i] = data[:, choices[i]]
    #Przynależnosc do grupy (na poczatku wszyscy do zero)
    C = np.zeros((1,N), dtype=np.int64)
    macierPrzynaleznosci = None

    iteration = 0
    groupsChanged = True
    while groupsChanged:
        iteration += 1
        #Interesuje nas minimalna wartość <r,r> -2<u,r>, gdzie u to wektor z danych a r to jakies centrum
        iloczynySkalarne = -2. * np.dot(data.T, R)
        R**=2
        kwadratyDlugosciR = np.sum(R, axis=0, keepdims=True)
        iloczynySkalarne += kwadratyDlugosciR

        #Dla kazdego wektora z danych wybieramy najblizszy wektor z R i aktualizujemy grupy
        newC = np.argmin(iloczynySkalarne, axis=1)
        groupsChanged = not np.array_equal(C, newC)
        C = newC

        #Obliczamy srodki ciezkosci dla kazdej grupy
        macierzPrzynaleznosci = np.take(I, C, axis=0)
        #Sumy danych w każdej z grup
        R = np.dot(data, macierzPrzynaleznosci)
        liczebnosciGrup = np.maximum(np.sum(macierzPrzynaleznosci, axis=0, keepdims=True), 1)
        R /= liczebnosciGrup

    #Liczymy wielkosci grup, odleglosci miedzy grupami, srednie i najwieksze odleglosci w grupach
    groupSizes = np.sum(macierzPrzynaleznosci, axis=0).astype(np.int64)
    interCluster = computeInterCluster(R)
    intraClusterAvg, intraClusterMax = computeIntraClusterDistances(data, C, R, I, K, N)
    
    if printInfo:
        print("Iterations:", iteration, "\n")
        print("Group sizes:\n", groupSizes, "\n")
        print("Inter cluster distances:")
        print(interCluster, "\n")
        print("Intra cluster average distance to center\n", intraClusterAvg, "\n")
        print("Intra cluster maximum distance to center\n", intraClusterMax, "\n")
    
    return (C, R, interCluster, intraClusterAvg, intraClusterMax)

In [ ]:
def generateData(N,d,K,p,mi,sigma):
    L = [np.linalg.cholesky(sigma[i]) for i in range(len(sigma))]
    choices = np.random.choice(np.arange(0,K), N, p=p)
    data = np.random.randn(d,N)
    
    #Jeśli X ~ N(mi, sigma) oraz Y = c + B*X, 
    #to Y ~ N(c + B * mi, B*sigma*B^T)
    #Jak wezmę X ~ N(zeros, I), c = mi oraz B = L, 
    #gdzie sigma = L*L^T, to otrzymam Y ~ N(mi + L*zeros, L*I*L^T) =
    # N(mi, L*L^T) = N(mi, sigma)
    
    for i in range(N):
        data[:,i] = mi[:,choices[i]] + np.dot(L[choices[i]], data[:,i])
        
    return data

In [ ]:
N = 1000000
d = 1000
K = 1000
p = np.ones(K) / K
q = 10
mi = np.repeat(q*np.arange(1,K+1)[np.newaxis], d, axis=0)
sigma = [np.eye(d) for i in range(1,K+1)]

data = generateData(N,d,K,p,mi,sigma)
